# Authors: Andrea ZANON, Hamza ZERHOUNI

In [1]:
using DataFrames
using JuMP, Gurobi

Create first basic model

In [4]:
# number of teams
N = 5
# number of weeks
W = N-1
# number of days per weeks
D = 7; # we suppose week starts on tuesday, beacause monday is actually part of the previous football week

# define mondays, tuesdays, wednesdays, thursdays, fridays, saturdays, sundays. Suppose we start playing on friday
#=
fr = [7*k + 1 for k in 0:18]
sa = [7*k + 2 for k in 0:18]
su = [7*k + 3 for k in 0:18]
mo = [7*k + 4 for k in 0:18]
tu = [7*k + 5 for k in 0:18]
we = [7*k + 6 for k in 0:18]
th = [7*k + 7 for k in 0:18]
;
=#

# list with elements of fr, sa, su, mo


We define the matrix $C \in R^{TxT}$, with $C_{ij} = \frac{1}{j-i}$

In [18]:
C = zeros((D*(W+1), D*(W+1)))

for t in 1:W*(D)
    for t_ in t+1:W*(D)
        C[t,t_] = 1/(t_ - t)
    end
end

In [19]:
5*7

35

In [23]:
C

28×28 Matrix{Float64}:
 0.0  1.0  0.5  0.333333  0.25      …  0.04       0.0384615  0.037037
 0.0  0.0  1.0  0.5       0.333333     0.0416667  0.04       0.0384615
 0.0  0.0  0.0  1.0       0.5          0.0434783  0.0416667  0.04
 0.0  0.0  0.0  0.0       1.0          0.0454545  0.0434783  0.0416667
 0.0  0.0  0.0  0.0       0.0          0.047619   0.0454545  0.0434783
 0.0  0.0  0.0  0.0       0.0       …  0.05       0.047619   0.0454545
 0.0  0.0  0.0  0.0       0.0          0.0526316  0.05       0.047619
 0.0  0.0  0.0  0.0       0.0          0.0555556  0.0526316  0.05
 0.0  0.0  0.0  0.0       0.0          0.0588235  0.0555556  0.0526316
 0.0  0.0  0.0  0.0       0.0          0.0625     0.0588235  0.0555556
 0.0  0.0  0.0  0.0       0.0       …  0.0666667  0.0625     0.0588235
 0.0  0.0  0.0  0.0       0.0          0.0714286  0.0666667  0.0625
 0.0  0.0  0.0  0.0       0.0          0.0769231  0.0714286  0.0666667
 ⋮                                  ⋱  ⋮                     
 0.0  0

Simple model, we are considering single league and making assumption that each team pla

In [32]:
model = Model(Gurobi.Optimizer)

# define variables
# 1 if team i plays team j on day d of week w, 0 otherwise
@variable(model, x[i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], Bin)

# define objective
# remember week begins on tuesday, this is why w_ goes from w+1
@objective(model, Min, sum(C[7*w + d, 7 * (w+1) + d_]* x[i, j, w, d] * x[i, k, w+1, d_] for i in 1:N, j = 1:N, k = 1:N, w = 0:W-2, d = 1:D, d_ = 1:D))

# define constraints
# each team can never play itself
@constraint(model, [i = 1:N, w = 0:W-1, d = 1:D], sum(x[i, i, w, d]) == 0)

# each team plays at most one team per week
@constraint(model, [i = 1:N, w = 0:W-1], sum(x[i, j, w, d] for j = 1:N, d = 1:D) <= 1)

# if team A plays team B, then team B plays team A
@constraint(model, [i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

# each team plays exactly N-1 games
@constraint(model, [i = 1:N], sum(x[i, j, w, d] for j = 1:N, w = 0:W-1, d = 1:D) == N-1)

# all teams need to play each other exactly once
@constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w = 0:W-1, d = 1:D) == 1)

# at least four days of rest (if they play monday, they can't play again until saturday)
@constraint(model, [i = 1:N, w = 0:W-2], sum(x[i, j, w+1, d1] for j = 1:N, d1 = 1:D) - sum(x[i, k, w, d2] for k = 1:N, d2 = 1:D) >= 4)

# can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
@constraint(model, [i = 1:N, j = 1:N, w = 0:W-1], sum(x[i, j, w, d] for d = 1:3) == 0)

# solve model
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1005 rows, 700 columns and 4710 nonzeros
Model fingerprint: 0x9c74593f
Model has 18375 quadratic objective terms
Variable types: 0 continuous, 700 integer (700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 140 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-callback calls 33, time in user-callback 0.00 sec
